In [2]:
!pip install arxiv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=65573958e9006edbbfd7c8e784edf716c622c3364a0695672a3a8b1a3d5834ce
  Stored in directory: /home/sourab/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [arxiv]


In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
!pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl


In [26]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_char_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [27]:
wiki.name

'wikipedia'

In [17]:
!pip install langchain_community

In [28]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x751d741dd250>, search_kwargs={})

In [29]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search","Search for information about LangSmith.for any information about LangSmith you should use this tool")

In [30]:
retriever_tool.name

'langsmith_search'

In [31]:
#Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_char_max=200)
arxiv= ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [46]:
tools=[wiki,arxiv,retriever_tool]

In [47]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/sourab/Desktop/GenAi/.venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='Search for information about LangSmith.for any information about LangSmith you should use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x751d581113a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<lan

In [48]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Ensure API key is set
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# 1. Define the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# # 2. Define the Prompt
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful research assistant."),
#         ("human", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )

In [49]:
from langchain import hub
#Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [50]:
#Agent
agent = create_tool_calling_agent(llm,tools,prompt)

In [53]:
#AgentExecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [54]:
agent_executor.invoke({"input":"Tell me something about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Ski

{'input': 'Tell me something about LangSmith',
 'output': "LangSmith is a platform for building production-grade LLM applications.  It offers observability features to analyze traces and configure metrics, dashboards, and alerts.  It also includes tools for evaluating application performance, collecting human feedback, and facilitating prompt engineering.  LangSmith is framework-agnostic and can be used with or without LangChain's open-source frameworks."}

In [55]:
agent_executor.invoke({"input":"Tell me something about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusi

{'input': 'Tell me something about Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence focused on developing statistical algorithms that learn from data and generalize to unseen data.  This allows them to perform tasks without explicit instructions.  Deep learning, a subfield of ML, uses neural networks to achieve high performance in various applications.  ML is used across many fields, including natural language processing, computer vision, speech recognition, and more.  The application of ML to business problems is called predictive analytics.  The foundations of ML are statistics and mathematical optimization methods.  Data mining, focusing on exploratory data analysis, is a related field.  Probably approximately correct learning provides a theoretical framework for describing machine learning.'}

In [56]:
agent_executor.invoke({"input":"prime minister of india"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'prime minister of india'}`


Page: Prime Minister of India
Summary: The prime minister of India (ISO: Bhārata kē Pradhānamantrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despite the president of India being the nominal head of the executive.  The prime minister has to be a member of one of the houses of bicameral Parliament of India, alongside heading the respective house. The prime minister and the cabinet are at all times responsible to the Lok Sabha.
The sitting prime minister ranks third in the Order of Precedence of India and is appointed by the president of India; however, the prime minister has to enjoy the confidence of the majority of Lok Sabha members, who are directly elected every five years, lest the prime minister shall resign. The prime minister can be a member of the Lok Sabha or the Rajya Sab

{'input': 'prime minister of india',
 'output': 'The current prime minister of India is Narendra Modi.  He has been in office since May 26, 2014.'}

In [58]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension.The paper 1605.08386, titled "Heat-bath random walks with Markov bases", published on May 26, 2016, by Caprice Stanley and Tobias Windisch, studies graphs on lattice points with edges determined by a finite set of allowed moves.  The authors bound the diameter of these graphs and analyze the mixing behavior of hea

{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper 1605.08386, titled "Heat-bath random walks with Markov bases", published on May 26, 2016, by Caprice Stanley and Tobias Windisch, studies graphs on lattice points with edges determined by a finite set of allowed moves.  The authors bound the diameter of these graphs and analyze the mixing behavior of heat-bath random walks on them, providing conditions for the walk to be an expander in fixed dimension.'}